# Modified 3D PMRNN

## Module Imports

In [ ]:
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=config)
import cv2
import os
import Utils
import numpy as np
import matplotlib.pyplot as plt
from CONFIGURATION import CONFIGURATION
from Reconstruction import Reconstruction

In [ ]:
print("TensorFlow version:", tf.__version__)
print("cv2 version:",cv2.__version__)
print("np version:",np.__version__)
print("plt version:",plt.__version__)

In [1]:
import sklearn
print("sklearn version:",sklearn.__version__)

sklearn version: 1.2.2


In [ ]:
from tensorflow.keras import layers,losses #,Sequential,metrics
from tensorflow.keras.models import Model
# from tensorflow.keras.layers import Layer
# from tensorflow.keras import optimizers 
from tensorflow.keras.optimizers import * 
# from tensorflow.keras.optimizers.experimental import SGD
# from tensorflow.image import ssim

## Load new Data

### reading data

In [ ]:
image_data = Utils.read_data("Berea_2d25um_binary.raw")
image_data.shape

### Voxel extraction 

In [ ]:
voxels = Utils.extract_subvolumes(image_data)
voxels.shape

In [ ]:
del image_data

### Train and test data extraction

In [ ]:
X_train = voxels[:-6]
X_test = voxels[-6:]

In [ ]:
del voxels

In [ ]:
len(X_train)

### Save Traited data for uses

In [ ]:
with open('X_train.npy', 'wb') as f:
    np.save(f, X_train)

In [ ]:
with open('X_test.npy', 'wb') as f:
    np.save(f, X_test)

## TensorFlow GPU setup

In [ ]:
device = tf.config.list_physical_devices("GPU")
print(device)
tf.config.experimental.set_memory_growth(device[0],True)

In [ ]:
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [ ]:
tf.keras.backend.clear_session()

## Model creation

### Build and Compile model

In [ ]:
reconstruction = Reconstruction()
reconstruction.summary()
reconstruction.compile()

## Load Traited data for training (voxels)

In [ ]:
X_train=np.load('X_train.npy')
X_train.shape

### idk

In [ ]:
learnedVoxels = []
inferenceVoxels = []
for voxel in X_train:
    learnedVoxels.append(voxel[:-1])
    inferenceVoxels.append(voxel[1:])

# learnedVoxels = np.array(learnedVoxels)
# inferenceVoxels = np.array(inferenceVoxels)

## Training 

### Imports

In [ ]:
from mealpy.swarm_based import GWO
from sklearn.preprocessing import LabelEncoder
import time
from datetime import datetime

### Lables

In [ ]:
OPT_ENCODER = LabelEncoder()
#OPT_ENCODER.fit(['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam'])
OPT_ENCODER.fit(['SGD', 'RMSprop', 'Adagrad', 'Adam'])

### Solution Decodation function

In [ ]:
def decode_solution(solution):
    batch_size = 2**int(solution[0])
    
    learning_rate = solution[1]
    
    opt_int = int(solution[2])
    opt = OPT_ENCODER.inverse_transform([opt_int])[0]
    
    epoch = 10 *int(solution[3])
    
    num_filters = int(solution[4])
    
    latent_space_dim = 2**int(solution[5])
    
    reduced_dim = 2**int(solution[6])
    
    return [batch_size,learning_rate,opt,epoch,num_filters,latent_space_dim,reduced_dim]

### Objective Function

In [ ]:
def objective_function(solution):
    tf.keras.backend.clear_session()
    batch_size,learning_rate,opt,epoch,num_filters,latent_space_dim ,reduced_dim = decode_solution(solution)
    reconstruction = Reconstruction(inputShape=CONFIGURATION["INPUT_SHAPE"],
                                     latent_space_dim = latent_space_dim,
                                     reducedDimension = reduced_dim,
                                     num_conv_layers = num_filters,
                                     learning_rate = learning_rate,
                                     batch_size = batch_size,
                                     epochs = epoch,
                                     opt = opt,
                                   )
#     reconstruction.summary()
    reconstruction.compile()
    
    globalStart=time.time()
    histories = []
    for i in range(len(learnedVoxels)):
        print(f"currently working one voxel : {i +1}, voxels left : {len(learnedVoxels)-(i+1)} ")
        start = time.time()
        print(f"Started at : {datetime.fromtimestamp(start)}")
        histories.append(reconstruction.train(learnedVoxels[i],inferenceVoxels[i]))
        end = time.time()
        print(f"Ended at : {datetime.fromtimestamp(start)} and it took {end-start}s")
    globalEnd=time.time()
    print(f"The global Training ended after : {globalEnd-globalStart}s")
    reconstruction.save(save_folder=f"model_{batch_size}_{learning_rate}_{opt}_{epoch}_{num_filters}_{latent_space_dim}_{reduced_dim}")
    
    return histories[-1].history['loss'][-1]

### Configurations

In [ ]:
LB = [3    , 0.001 , 0   , 2      ,  4  ,  6    , 7 ]
UB = [5.99 , 0.01  , 3.99, 4.99  ,  8.99,  8.99 , 9.99 ]

problem_dict = {
    "fit_func": objective_function,
    "lb": LB,
    "ub": UB,
    "minmax": "min",
    "verbose":True,
}
metaheuristic_configuration={
    "epoch":1,
    "pop_size":50
}

### Lunching the metaheuristic

In [ ]:
model = GWO.BaseGWO(problem_dict,epoch=metaheuristic_configuration["epoch"],pop_size=metaheuristic_configuration["pop_size"])

In [ ]:
model.solve()

In [ ]:
print(f"Best solution : {model.solution[0]}")
batch_size,learning_rate,opt,epoch,num_filters,latent_space_dim ,reduced_dim = decode_solution(model.solution[0])
print(f"Batch size : {batch_size}, learning_rate : {learning_rate}, opt : {opt}, epoch : {epoch}, num_filters : {num_filters}, latent_space_dim : {latent_space_dim}, reduced_dim : {reduced_dim}")
model.history.save_global_objectives_chart(filename="hello/goc")
model.history.save_local_objectives_chart(filename="hello/loc")

model.history.save_global_best_fitness_chart(filename="hello/gbfc")
model.history.save_local_best_fitness_chart(filename="hello/lbfc")

model.history.save_runtime_chart(filename="hello/rtc")

model.history.save_exploration_exploitation_chart(filename="hello/eec")

model.history.save_diversity_chart(filename="hello/dc")

model.history.save_trajectory_chart(list_agent_idx=[3, 5], selected_dimensions=[3], filename="hello/tc")

## Training the model

In [ ]:
histories = []
for i in range(3):
    print(f"currently working one voxel : {i +1}, voxels left : {3-(i+1)} ")
    histories.append(reconstruction.train(learnedVoxels[i],inferenceVoxels[i]))

In [ ]:
saved = Reconstruction.load(save_folder=f"test")

## Testing the model

In [ ]:
first  = X_test[0]
Topredict = first[1:]
test = first[:-1]

In [ ]:
testing = []
inputLayer = first[0]
# for i in range(len(first)-1):
for i in range(10):

    learned = reconstruction.learnedPrior(inputLayer.reshape(1,256,256,1))
    testing.append(reconstruction.Generate.predict([inputLayer.reshape(1,256,256,1),learned],steps=1))
    inputLayer = testing[-1]

In [ ]:
len(testing)

In [ ]:
n = 10

plt.figure(figsize=(20,4))
for i in range(n):
    # original
    ax = plt.subplot(2,n,i+1)
    plt.imshow(Topredict[i].astype("float32"))
    plt.title('original')
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # reconstructed
    ax = plt.subplot(2,n,i+1+n)
    plt.imshow(testing[i].reshape(256,256,1))
    plt.title('reconstructed')
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

In [ ]:
# reconstruction.Generate.save('reconstruction_10batch_10epochs.h5')

In [ ]:
file_path = "reconstructed.raw"
testing.insert(0, first[0].reshape(1,256,256,1))
print(len(testing))

In [ ]:
final = np.array(testing,np.float32).reshape(250,256,256)
final.shape

In [ ]:
file_path = "my_array.npy"
np.save("file", final)

In [ ]:
final.tofile(file_path)

In [ ]:
Volume = np.fromfile(file_path, dtype=np.uint8)
Volume = Volume.reshape(250,256,256)
Volume.shape